In [1]:
from rank_bm25 import BM25Okapi
from pymongo import MongoClient
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()


In [47]:
user_input = 'Great'

In [48]:
client = MongoClient('mongodb://localhost:27017/')
# select a database
db = client['inverted_index']
mycollection = db['inverted_index']

In [49]:
words = user_input.split()

In [50]:
words

['Great']

In [51]:
rel_docs = set()
for word in words:
    myquery = {"word": word}
    results  = mycollection.find(myquery)
    for result in results:
        docs = result['doc_ids']
        print(word,docs)
        rel_docs.update(docs)

Great [192, 255, 299, 320, 401, 423, 471, 480, 689, 753, 797, 818, 922, 969, 978, 1188, 1251, 1295, 1417, 1858]


In [52]:
rel_docs = list(rel_docs)

In [53]:
len(rel_docs)

20

In [54]:
documents  = []

for idx in rel_docs:
    query = f"SELECT * FROM sites where id = {idx};"
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        doc_id = row[0]
        url = row[1]
        text = row[2]
        documents.append(text)

In [55]:
len(documents)

20

In [56]:
tokenized_corpus = [doc.split(" ") for doc in documents]

In [57]:
bm25 = BM25Okapi(tokenized_corpus)


In [58]:
tokenized_query = words

In [59]:
doc_scores = bm25.get_scores(tokenized_query)

In [60]:
doc_scores.argsort()[::-1]

array([12, 16, 14,  1,  4,  7,  9, 11, 18, 19, 17,  2, 13, 15,  6,  3, 10,
        8,  5,  0], dtype=int64)

In [61]:
max_scored = doc_scores.argsort()[::-1][0]
print(max_scored)

12


In [62]:
documents[max_scored]

'Scientists reveal hidden corridor in Great Pyramid of Giza CAIRO, March 2 (Reuters) - A hidden corridor nine metres (30 feet) long has been discovered close to the main entrance of the 4,500-year-old Great Pyramid of Giza, and this could lead to further findings, Egyptian antiquities officials said on Thursday. The discovery within the pyramid, the last of the Seven Wonders of the Ancient World still standing, was made under the Scan Pyramids project that since 2015 has been using non-invasive technology including infrared thermography, 3D simulations and cosmic-ray imaging to peer inside the structure. An article published in the journal Nature on Thursday said the discovery could contribute to knowledge about the construction of the pyramid and the purpose of a gabled limestone structure that sits in front of the corridor. The Great Pyramid was constructed as a monumental tomb around 2560 BC during the reign of the Pharaoh Khufu, or Cheops. Built to a height of 146 metres (479 feet)

In [18]:
doc_id = rel_docs[max_scored]

In [19]:
query = f"SELECT * FROM sites where id = {doc_id};"
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

(630, 'https://www.bbc.com/news/articles/cn4792z2n6eo', 'Thousands more to lose NHS dentist access Access to NHS services in the region has been described as "extremely limited" At a glance Thousands more patients in Dumfries and Galloway face being deregistered from NHS dental services About 5,000 people will be affected by the move at the Blue Door Practice in Dumfries It follows similar action at a number of other sites across the region NHS Dumfries and Galloway said it was working hard to tackle the situation Thousands more patients are to be deregistered from NHS dental services in the south of Scotland. NHS Dumfries and Galloway said it had received notification of the move by the Blue Door Dental Practice. It means that about 5,000 adults face being deregistered by the Dumfries facility from 1 June. With access to NHS registration in the region "extremely limited" they are likely to have to travel outside the region to secure such provision or have to go private. Read more stor

In [20]:
documents[3]

'Thousands more to lose NHS dentist access Access to NHS services in the region has been described as "extremely limited" At a glance Thousands more patients in Dumfries and Galloway face being deregistered from NHS dental services About 5,000 people will be affected by the move at the Blue Door Practice in Dumfries It follows similar action at a number of other sites across the region NHS Dumfries and Galloway said it was working hard to tackle the situation Thousands more patients are to be deregistered from NHS dental services in the south of Scotland. NHS Dumfries and Galloway said it had received notification of the move by the Blue Door Dental Practice. It means that about 5,000 adults face being deregistered by the Dumfries facility from 1 June. With access to NHS registration in the region "extremely limited" they are likely to have to travel outside the region to secure such provision or have to go private. Read more stories from the south of Scotland Listen to news from Dumfr